In [1]:
import pandas as pd
import numpy as np

import glob
import tqdm
import gc

print('>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!')
f'pd : {pd.__version__}  |  np : {np.__version__}  |  tqdm : {tqdm.__version__}'

>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!


'pd : 1.0.5  |  np : 1.16.5  |  tqdm : 4.47.0'

## 셀-행정동 매핑자료 로드 (서울특별시 전체)

In [2]:
region_name_map = pd.read_csv('2_ADMDONG_CD-ADMDONG_NM.csv')
region_name_map = region_name_map[(region_name_map.sido_nm == '서울특별시')].reset_index(drop=True).drop(columns=['sido_cd', 'sido_nm'])

display(region_name_map.head())
print('서울특별시 소속 50셀 코드 수 :', region_name_map.shape[0])
print(f'서울특별시 소속 행정동 코드 (상위 5개) : {list(region_name_map.admdong_cd)[:5]} ...')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm
0,11110,종로구,11110540,삼청동
1,11110,종로구,11110615,종로1.2.3.4가동
2,11110,종로구,11110650,혜화동
3,11110,종로구,11110680,창신2동
4,11110,종로구,11110700,숭인1동


서울특별시 소속 50셀 코드 수 : 424
서울특별시 소속 행정동 코드 (상위 5개) : [11110540, 11110615, 11110650, 11110680, 11110700] ...


In [3]:
cell_code_map = pd.read_csv('1_50CELL-ADMDONG_CD.csv').drop(columns=['x_axis', 'y_axis'])
cell_code_map = cell_code_map[cell_code_map.admdong_cd.isin(region_name_map.admdong_cd)]

display(cell_code_map.head())
print('서울특별시 소속 50셀 코드 수 :', cell_code_map.shape[0])
print(f'서울특별시 소속 50셀 코드 (상위 5개) : {list(cell_code_map.id)[:5]} ...')

,id,admdong_cd
757529,46047699,11500620
758464,46059888,11500620
758465,46059889,11500620
759402,46072077,11500620
759403,46072078,11500620


서울특별시 소속 50셀 코드 수 : 242363
서울특별시 소속 50셀 코드 (상위 5개) : [46047699, 46059888, 46059889, 46072077, 46072078] ...


## 행정동별 월별 유동인구 총합 계산 (1월만)

In [4]:
# filename = 'SEOUL_201901*.csv'
# file_list = glob.glob(filename)
# year = int(filename[6:10])
# month = int(filename[10:12])

# df_list = []
# for file in file_list:
#     tmp_df = pd.read_csv(file, usecols=['id', 'total'])
#     tmp_df = pd.merge(tmp_df, cell_code_map, how='left', on='id').groupby('admdong_cd')['total'].sum().reset_index()
#     df_list.append(tmp_df)
#     pass

# month_df = pd.merge(df_list[0], df_list[1], how='outer', on='admdong_cd')
# month_df = pd.merge(month_df, df_list[2], how='outer', on='admdong_cd')
# month_df['year'] = year
# month_df['month'] = month
# month_df['total'] = month_df[['total_x', 'total_y', 'total']].sum(axis=1)

# month_df = month_df[['admdong_cd', 'year', 'month', 'total']]
# month_df = region_name_map.merge(month_df, how='left', on='admdong_cd')
# month_df

## 행정동별 월별 유동인구 총합 계산 (19년 1월-4월, 20년 1-4월)

In [5]:
result_list = []

for year in [2019, 2020]:
    for month in tqdm.tqdm_notebook(range(1, 5)):
        month_str = f'0{month}'[-2:]
        filename = f'SEOUL_{year}{month_str}*.csv'
        file_list = glob.glob(filename)

        df_list = []
        for file in file_list:
            tmp_df = pd.read_csv(file, usecols=['id', 'total'])
            tmp_df = pd.merge(tmp_df, cell_code_map, how='left', on='id').groupby('admdong_cd')['total'].sum().reset_index()
            df_list.append(tmp_df)
            pass

        month_df = pd.merge(df_list[0], df_list[1], how='outer', on='admdong_cd')
        month_df = pd.merge(month_df, df_list[2], how='outer', on='admdong_cd')
        month_df['year'] = year
        month_df['month'] = month
        month_df['total'] = month_df[['total_x', 'total_y', 'total']].sum(axis=1)

        month_df = month_df[['admdong_cd', 'year', 'month', 'total']]
        month_df = region_name_map.merge(month_df, how='left', on='admdong_cd')
        
        result_list.append(month_df)
        gc.collect()

C:\Users\dslab01\anaconda3\envs\gis_prac\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


## 최종 데이터프레임 결측치 확인 및 저장

In [6]:
result_df = pd.concat(result_list, ignore_index=True)
result_df

,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,total
0,11110,종로구,11110540,삼청동,2019,1,5881315
1,11110,종로구,11110615,종로1.2.3.4가동,2019,1,61377525
2,11110,종로구,11110650,혜화동,2019,1,16015862
3,11110,종로구,11110680,창신2동,2019,1,2451402
4,11110,종로구,11110700,숭인1동,2019,1,7420365
...,...,...,...,...,...,...,...
3387,11710,송파구,11710710,잠실6동,2020,4,31576430
3388,11740,강동구,11740540,명일2동,2020,4,8103775
3389,11740,강동구,11740590,암사3동,2020,4,9953712
3390,11740,강동구,11740610,천호2동,2020,4,23695173


In [11]:
result_df.isna().sum()

sgg_cd        0
sgg_nm        0
admdong_cd    0
admdong_nm    0
year          0
month         0
total         0
dtype: int64

In [9]:
result_df.to_pickle('서울특별시_월별_행정동별_총합.pkl')